<a href="https://colab.research.google.com/github/BenShieh233/Learn_Python/blob/main/Assignment%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set-Up

In [1]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo. 
#Uncomment the appropriate solver that you need.
#for reference, see https://colab.research.google.com/drive/1yGk8RB5NXrcx9f1Tb-oCiWzbxh61hZLI?usp=sharing

#installing and importing pyomo
!pip install -q pyomo
from pyomo.environ import *

###installing and importing specific solvers (uncomment the one(s) you need)
###glpk
!apt-get install -y -qq glpk-utils
###cbc
#!apt-get install -y -qq coinor-cbc
###ipopt
#!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
#!unzip -o -q ipopt-linux64
###bonmin
#!wget -N -q "https://ampl.com/dl/open/bonmin/bonmin-linux64.zip"
#!unzip -o -q bonmin-linux64
###couenne
#!wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip"
#!unzip -o -q couenne-linux64
###geocode
#!wget -N -q "https://ampl.com/dl/open/gecode/gecode-linux64.zip"
#!unzip -o -q gecode-linux64

#Using the solvers:
#SolverFactory('glpk', executable='/usr/bin/glpsol')
#SolverFactory('cbc', executable='/usr/bin/cbc')
#SolverFactory('ipopt', executable='/content/ipopt')
#SolverFactory('bonmin', executable='/content/bonmin')
#SolverFactory('couenne', executable='/content/couenne')
#SolverFactory('gecode', executable='/content/gecode')

     |████████████████████████████████| 11.1 MB 13.6 MB/s 
     |████████████████████████████████| 49 kB 5.5 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 124015 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting u

In [2]:
import pandas as pd

# Load the data

In [4]:
df_DC = pd.read_excel('/content/P4_DCLocation.xlsx', sheet_name= 'DCdata')
df_city = pd.read_excel('/content/P4_DCLocation.xlsx', sheet_name = 'citydata')
df_cost = pd.read_excel('/content/P4_DCLocation.xlsx', sheet_name = 'distcost')

In [5]:
# Converting distance cost data to a list of list
distcost = df_cost.iloc[:,1:].values.tolist()

In [6]:
# Reading in DC's fixed costs
capacity = df_DC['capacity'].tolist()
fixed_cost = df_DC['fixedcost'].tolist()
DCs = df_DC['location'].tolist()

In [7]:
# Reading in cities' demands
demand = df_city['demand'].tolist()
cities = df_city['city'].tolist()

In [8]:
num_DCs = len(DCs)
num_cities = len(cities)

In [9]:
from re import I
#define the optimization model
model = ConcreteModel()

# DVs
model.x = Var(range(num_DCs), domain = Binary)
model.y = Var(range(num_DCs), range(num_cities), domain = NonNegativeReals)

#Objective: minimize total costs
fixed_costs = sum(model.x[i] * fixed_cost[i] for i in range(num_DCs))
variable_costs = sum(model.y[j,i]*distcost[j][i] for i in range(num_cities) for j in range(num_DCs))
total_costs = fixed_costs + variable_costs
model.Objective = Objective(expr = total_costs, sense = minimize)

# satisying each city's demand
model.citydemand = ConstraintList()
for i in range(num_cities):
  model.citydemand.add(sum(model.y[j,i] for j in range(num_DCs)) >= demand[i])

# satisfying each DC's capacity
model.capacity = ConstraintList()
for j in range(num_DCs):
  model.capacity.add(sum(model.y[j,i] for i in range(num_cities)) <= capacity[j]*model.x[j])

In [10]:
model.pprint()

6 Set Declarations
    capacity_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   23 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23}
    citydemand_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   76 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76}
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   23 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22}
    y_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain            

In [11]:
#solve the model
opt = SolverFactory('glpk')
opt.options['tmlim'] = 5 #specifies the time limit (in seconds)
opt.options['mipgap'] = .01 #specifies the optimality gap tolerance (.01 means can stop if <1% of optimal obj)
results = opt.solve(model, tee=True) #can set tee=True if you want to see the details.

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --tmlim 5 --mipgap 0.01 --write /tmp/tmpqbwcgkw0.glpk.raw --wglp /tmp/tmpkdnhqyrc.glpk.glp
 --cpxlp /tmp/tmp_xvfu_w5.pyomo.lp
Reading problem data from '/tmp/tmp_xvfu_w5.pyomo.lp'...
/tmp/tmp_xvfu_w5.pyomo.lp:7371: warning: lower bound of variable 'x1' redefined
/tmp/tmp_xvfu_w5.pyomo.lp:7371: warning: upper bound of variable 'x1' redefined
100 rows, 1772 columns, 3520 non-zeros
23 integer variables, all of which are binary
7394 lines were read
Writing problem data to '/tmp/tmpkdnhqyrc.glpk.glp'...
9015 lines were written
GLPK Integer Optimizer, v4.65
100 rows, 1772 columns, 3520 non-zeros
23 integer variables, all of which are binary
Preprocessing...
99 rows, 1771 columns, 3519 non-zeros
23 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  4.975e+04  ratio =  4.975e+04
GM: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
EQ: min|aij| =  1.000e+00  m

In [15]:
#print the results
DC_location_solutions = [model.x[i]() for i in range(num_DCs)]
distribution_solutions = [[model.y[j,i]() for i in range(num_cities)] for j in range(num_DCs)]

In [17]:
df_dc_sol = pd.DataFrame(DC_location_solutions, index = DCs)

In [18]:
df_dc_sol

,0
Durham,0.0
Fort Wayne,0.0
St. Petersburg,0.0
Laredo,0.0
Buffalo,0.0
Madison,0.0
Lubbock,1.0
Chandler,0.0
Scottsdale,0.0
Reno,0.0


In [21]:
print("Total Cost:", model.Objective())

print("DC locations:")
for i in range(num_DCs):
  if model.x[i]()==1:
    print(DCs[i])

Total Cost: 58006498.0
DC locations:
Lubbock
Glendale
Winston–Salem
Irving
Fremont
